<img src="py-logo.png" width="100pt"/>

# INTRODUCTION TO PYTHON 
# II – WORKING WITH DATA
*Lasse Ruokolainen*

*Seasoned Data Master, BILOT Consulting Oy* 

***

## (1) Functions and methods
Python has two ways of operating on data; **function**s and **method**s. Functions take data as input (and potentially additional arguments), whereas methods do a predefined operation *within* an object. Syntax: `function(input,*arguments)`, `object.method()`. Some objects also have **attribute**s, such as `shape` of arrays and dataframes, which work just like **method**s, except that no brackets are used.  

### (a) *Functions*

In [2]:
from numpy.random import uniform as runif

# generate example data, using a function:
x = runif(size=10)
print(x) 

[0.46027564 0.57175792 0.674136   0.93563297 0.80950921 0.60648601
 0.06768172 0.52367973 0.67305902 0.17534367]


In [3]:
# Inbuilt function:
print(max(x))
print(min(x))
print(sum(x))

0.935632967258957
0.06768171616504237
5.49756189375235


In [6]:
# define a custom function:
def my_mean(data):
    """
    This function returns the mean value of 
    input numeric data, using inbuilt functions
    sum() and len().
    """
    return sum(data)/len(data)

# use the function:
print(my_mean(x))
print('%.3f' %my_mean(x))

import numpy as np
print(np.mean(x))

0.549756189375235
0.550
0.549756189375235


### (b) *Methods*

In [7]:
import pandas as pd

# make x a pandas Series
x2 = pd.Series(x)
print(x2)

0    0.460276
1    0.571758
2    0.674136
3    0.935633
4    0.809509
5    0.606486
6    0.067682
7    0.523680
8    0.673059
9    0.175344
dtype: float64


In [8]:
# use method for Series:
print(x2.mean())
print(x2.max())
print(x2.prod())

0.549756189375235
0.935632967258957
0.0003408836523988479


In [12]:
# make a dataframe:
df = pd.DataFrame({
    'x' : x,
    'x_squared' : x**2
})

# use methods on dataframe:
print(df.mean(),'\n')
print(df.apply(my_mean))

x            0.549756
x_squared    0.365433
dtype: float64 

x            0.549756
x_squared    0.365433
dtype: float64


How does one then know whether one is dealing with a **function**, **method**, or **attribute**?
Here the function `type()` comes handy:

In [13]:
print(type(df.shape))
print(type(df.sum))
print(type(df.index))
print(type(sum))

<class 'tuple'>
<class 'method'>
<class 'pandas.core.indexes.range.RangeIndex'>
<class 'builtin_function_or_method'>


***
## (2) Data operations and manipulations
It is typical that at least 80% of time in a project go to data handling and monipulation. Thus, it might be a good idea to know that is done 

### (a) *Read and inspect data*
In order to do something usefull in Python, one often needs to bring in data. If the data resides in a flat file, a convenient method is to use the `read_csv()` function in `Pandas`.

In [14]:
# read in a data set:
df = pd.read_csv('Datasets/tips.csv',index_col = 0)

df.head() # note: use of .method()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


Now that we have the data, below we'll go through a couple of handy methods. 

In [15]:
# query variable types:
print(df.dtypes)

total_bill    float64
tip           float64
sex            object
smoker         object
day            object
time           object
size            int64
dtype: object


In [16]:
# change data type:
df.sex = df.sex.astype('category')

In [17]:
# descriptive statistics:
df.describe(include = 'all') # note: use of .method()

,total_bill,tip,sex,smoker,day,time,size
count,244.000000,244.000000,244,244,244,244,244.000000
unique,NaN,NaN,2,2,4,2,NaN
top,NaN,NaN,Male,No,Sat,Dinner,NaN
freq,NaN,NaN,157,151,87,176,NaN
mean,19.785943,2.998279,NaN,NaN,NaN,NaN,2.569672
std,8.902412,1.383638,NaN,NaN,NaN,NaN,0.951100
min,3.070000,1.000000,NaN,NaN,NaN,NaN,1.000000
25%,13.347500,2.000000,NaN,NaN,NaN,NaN,2.000000
50%,17.795000,2.900000,NaN,NaN,NaN,NaN,2.000000
75%,24.127500,3.562500,NaN,NaN,NaN,NaN,3.000000


In [19]:
# tabulate the number of smokers: 
tab = df.pivot_table(
    index = 'day',
    columns = 'sex',
    values = 'smoker',
    aggfunc = 'count'
)
print(tab)

# convert to proportions:
print('\n',tab.apply(lambda x: x/sum(x),axis='rows').round(2))

sex   Female  Male
day               
Fri        9    10
Sat       28    59
Sun       18    58
Thur      32    30

 sex   Female  Male
day               
Fri     0.10  0.06
Sat     0.32  0.38
Sun     0.21  0.37
Thur    0.37  0.19


### (b) *Data*
What if you need to calculate new variables to the data or perform aggregations? 

In [21]:
# calculate new variable:
df['relative_tip'] = df.tip/df.total_bill
print(df.relative_tip.mean().round(3))

0.161


In [22]:
import numpy as np
# perform operation within categories by using groupby:
df2 = df.groupby(['day','time','smoker']).apply(np.mean)

In [23]:
# index the groupped dataframe, notice the double index!
df2.loc['Fri','size']
df2

total_bill       tip      size  relative_tip
day  time   smoker                                              
Fri  Dinner No       19.233333  2.750000  2.000000      0.139622
            Yes      19.806667  3.003333  2.222222      0.165347
     Lunch  No       15.980000  3.000000  3.000000      0.187735
            Yes      12.323333  2.280000  1.833333      0.188937
Sat  Dinner No       19.661778  3.102889  2.555556      0.158048
            Yes      21.276667  2.875476  2.476190      0.147906
Sun  Dinner No       20.506667  3.167895  2.929825      0.160113
            Yes      24.120000  3.516842  2.578947      0.187250
Thur Dinner No       18.780000  3.000000  2.000000      0.159744
     Lunch  No       17.075227  2.666364  2.500000      0.160311
            Yes      19.190588  3.030000  2.352941      0.163863

### (c) *Missing values*


In [24]:
mpg = pd.read_csv('Datasets/mpg.csv',index_col=0)
mpg.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,car name
0,18.0,8,307.0,130.0,3504.0,12.0,70.0,1,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693.0,11.5,70.0,1,buick skylark 320
2,18.0,8,318.0,150.0,3436.0,11.0,70.0,1,plymouth satellite
3,16.0,8,304.0,150.0,3433.0,12.0,70.0,1,amc rebel sst
4,17.0,8,302.0,140.0,3449.0,10.5,70.0,1,ford torino


Another good method for data inspection is `.info()`, which shows you not only the data dimensions, variable types, but also the number of missing values:

In [25]:
mpg.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 398 entries, 0 to 397
Data columns (total 9 columns):
mpg             398 non-null float64
cylinders       398 non-null int64
displacement    398 non-null float64
horsepower      392 non-null float64
weight          392 non-null float64
acceleration    398 non-null float64
model year      398 non-null float64
origin          398 non-null int64
car name        398 non-null object
dtypes: float64(6), int64(2), object(1)
memory usage: 31.1+ KB


OK, more precisely, you get the number of non-missing values. Still, we can easily calculate how many are missing:

In [26]:
mpg.isnull().sum()

mpg             0
cylinders       0
displacement    0
horsepower      6
weight          6
acceleration    0
model year      0
origin          0
car name        0
dtype: int64

In order to work with such data with missing values, we need to either remove those rows with incomplete records, or fill the blanks with something.

In [27]:
# drop missing values:
mpg.dropna().info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 392 entries, 0 to 397
Data columns (total 9 columns):
mpg             392 non-null float64
cylinders       392 non-null int64
displacement    392 non-null float64
horsepower      392 non-null float64
weight          392 non-null float64
acceleration    392 non-null float64
model year      392 non-null float64
origin          392 non-null int64
car name        392 non-null object
dtypes: float64(6), int64(2), object(1)
memory usage: 30.6+ KB


Dropping the rows with missing data obviously leads to, in this case, 6 rows less. The above command needs to be stored to a new dataframe, or setting `inplace=True` in the `.dropna()` method.

In [28]:
# fill na's with a value:
mpg.fillna(0).info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 398 entries, 0 to 397
Data columns (total 9 columns):
mpg             398 non-null float64
cylinders       398 non-null int64
displacement    398 non-null float64
horsepower      398 non-null float64
weight          398 non-null float64
acceleration    398 non-null float64
model year      398 non-null float64
origin          398 non-null int64
car name        398 non-null object
dtypes: float64(6), int64(2), object(1)
memory usage: 31.1+ KB


Here the missing valyes are replaced with the value `0`. Again, the result needs to be stored to a new dataframe, or setting `inplace=True` in the `.dropna()` method.

In [29]:
# fill na's with a value:
mpg.fillna(mpg.mean()).describe().round()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin
count,398.0,398.0,398.0,398.0,398.0,398.0,398.0,398.0
mean,24.0,5.0,193.0,104.0,2978.0,53.0,75.0,3.0
std,8.0,2.0,104.0,38.0,843.0,309.0,8.0,9.0
min,9.0,3.0,68.0,46.0,1613.0,8.0,14.0,1.0
25%,18.0,4.0,104.0,76.0,2228.0,14.0,73.0,1.0
50%,23.0,4.0,148.0,95.0,2832.0,16.0,76.0,1.0
75%,29.0,8.0,262.0,125.0,3608.0,17.0,79.0,2.0
max,47.0,8.0,455.0,230.0,5140.0,3035.0,82.0,82.0


In this case the missing values are replaced by the averages by each variable. Isn't that clever?